In [5]:
import cv2
import tensorflow as tf
import numpy as np
import os
import csv
import datetime

# --- Configuration ---
MODEL_PATH = 'face_model_cnn.keras'
LABELS_PATH = 'labels.npy'
ATTENDANCE_FILE = 'attendance_cnn.csv'
IMG_SIZE = (100, 100) # Must be the same size as trained on
CONFIDENCE_THRESHOLD = 70 # We can be more confident with a better model
# --- End Configuration ---

logged_names_today = set()

# --- Attendance Functions (Unchanged) ---
def load_attendance():
    global logged_names_today
    today_str = datetime.date.today().strftime('%Y-%m-%d')
    try:
        with open(ATTENDANCE_FILE, mode='r', newline='', encoding='utf-8') as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader, None)
            for row in csv_reader:
                if len(row) >= 2 and row[1] == today_str:
                    logged_names_today.add(row[0])
        print(f"Loaded {len(logged_names_today)} names already logged today: {logged_names_today}")
    except FileNotFoundError:
        print("Attendance file not found. A new one will be created.")
    except Exception as e:
        print(f"Error loading attendance: {e}")

def mark_attendance(name):
    if name not in logged_names_today:
        now = datetime.datetime.now()
        date_str = now.strftime('%Y-%m-%d')
        time_str = now.strftime('%H:%M:%S')
        try:
            with open(ATTENDANCE_FILE, mode='a', newline='', encoding='utf-8') as file:
                csv_writer = csv.writer(file)
                if os.path.getsize(ATTENDANCE_FILE) == 0:
                    csv_writer.writerow(['Name', 'Date', 'Time'])
                csv_writer.writerow([name, date_str, time_str])
            logged_names_today.add(name)
            print(f"Attendance marked for {name} at {time_str}")
        except Exception as e:
            print(f"Error writing to attendance file: {e}")

# --- Main Program Setup ---

# 1. Load the trained model
print("Loading trained model...")
if not os.path.exists(MODEL_PATH):
    print(f"Error: Model file not found at {MODEL_PATH}")
    print("Please run '03_model_trainer.py' first.")
    exit()
model = tf.keras.models.load_model(MODEL_PATH)

# 2. Load the label mapping
print("Loading labels...")
if not os.path.exists(LABELS_PATH):
    print(f"Error: Labels file not found at {LABELS_PATH}")
    print("Please run '03_model_trainer.py' first.")
    exit()
labels = np.load(LABELS_PATH, allow_pickle=True)
print(f"Model loaded. Recognizing for: {labels}")

# 3. Load *TWO* classifiers
try:
    frontal_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    profile_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
except Exception:
    print("Error loading Haar Cascade classifiers.")
    exit()

# 4. Load any existing attendance logs for today
load_attendance()

# 5. Start the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Starting webcam... Press 'q' to quit.")

# --- Main Loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break
       
    # We detect on grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # We predict on color
   
    # --- Detect both frontal and profile faces ---
    frontal_faces = frontal_face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))
    profile_faces = profile_face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(50, 50))

    if len(frontal_faces) > 0 and len(profile_faces) > 0:
        faces = np.concatenate((frontal_faces, profile_faces))
    elif len(frontal_faces) > 0:
        faces = frontal_faces
    elif len(profile_faces) > 0:
        faces = profile_faces
    else:
        faces = []
   
    for (x, y, w, h) in faces:
        # --- Preprocessing for the Model ---
       
        # 1. --- NEW: Crop the face from the *COLOR* frame ---
        face_crop = frame[y:y+h, x:x+w]
       
        # 2. Resize to the model's expected input size
        face_resized = cv2.resize(face_crop, IMG_SIZE)
       
        # 3. --- NEW: Normalize pixel values (0-1) ---
        face_normalized = face_resized.astype('float32') / 255.0
       
        # 4. --- NEW: Reshape for the model: (1, height, width, 3) ---
        face_reshaped = np.reshape(face_normalized, (1, IMG_SIZE[0], IMG_SIZE[1], 3))

        # --- Prediction ---
       
        # 5. Get the model's prediction
        prediction = model.predict(face_reshaped, verbose=0)
       
        # 6. Find the highest probability
        class_index = np.argmax(prediction)
        confidence = prediction[0][class_index] * 100
       
        # 7. Get the name from the labels file
        name = labels[class_index]
       
        # --- Display Results & Log Attendance ---
       
        if confidence > CONFIDENCE_THRESHOLD:
            display_text = f"{name} ({confidence:.2f}%)"
            color = (0, 255, 0) # Green
            mark_attendance(name)
        else:
            display_text = "Unknown"
            color = (0, 0, 255) # Red
           
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, display_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Show the final frame
    cv2.imshow('Live Face Recognizer', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Cleanup ---
cap.release()
cv2.destroyAllWindows()
print("Program stopped.")

Loading trained model...
Loading labels...
Model loaded. Recognizing for: ['Navraj' 'Rajni Rajisthani aunty' 'akriti' 'aneesh' 'ankush udham'
 'anshika' 'aromal' 'arsh dhir' 'aryan' 'asit' 'athul' 'avneet' 'bhuvi'
 'dhiraj' 'furquan' 'gautm' 'gautum' 'harjot' 'harsh' 'harsh pandey'
 'karan' 'keshav' 'krish' 'kriti' 'melwin' 'mohita' 'pavan' 'payal'
 'preetkaur walia' 'prince' 'rishi' 'riya' 'saksham' 'samarth' 'sanya'
 'sidak' 'srijiti' 'sujal' 'suman rani' 'vaibhav balu' 'varnan' 'vedant'
 'yashika' 'yogesh']
Loaded 5 names already logged today: {'payal', 'vedant', 'varnan', 'Ishir', 'sanya'}
Starting webcam... Press 'q' to quit.
Program stopped.
